## Heart Rate Processing and Baseline Correction 
Script by Nadu Barbashova 

This Script is also on Github in my Psychophysiology Folder: 

https://github.com/n-barbashova/psychophysiology/tree/master/HeartRate 


### About this script: 
My Experiment: 
Participants wait 60 seconds for stimuli (shocks or stimulations). In this time they watch a 60 second coundown. For 30 of these seconds they complete the flanker task and for the other 30 seconds they just watch the countdown. The order depends on which countdown it is. There are 36 countdowns, divided into 9 runs. 

The goal is to see how the heartrate is affected by the condition (is this a threat countdown (leading to shock) or a no-threat countdown (leading to stim)). I want to also see how heart rate is affected when it's the flanker task epoch vs the countdown epoch. This labels all the relevant info.

It also downsamples one additional time (by a factor of 10). This means that at the end of processing the sampling rate is 10hz. 

### What this script does:
* Adds Event Codes 
* Processes HR Data 
* Baseline Corrects 

This script takes the pre-processed ECG data and further processes it using functions from Neurokit. It cleans the heart rate, finds event codes and using the event codes, labels which condition this is (shock or stim), which interval it is (flanker or countdown). In addition to processing the heart rate it also does one second heart rate baseline correction, comparing the heart rate in the given interval to the heart rate in the one second before the countdown began. 

It adds event code info based on the number in the "EVENT" col, and it adds an intervalNum col to keep track of which countdown per run this is (each run has 4 countdowns). 
 
### Input required: 
Timings files (txt files) with info on the subject and run number in the filename. 
Example: 110_run9.txt (This is for subject 100, run 9) 

### Output from this script: 
The first main block/cell of this script will output a new processed file for each input file. 

The second main block of this script will combine all subjects and runs together and add a column for countdown number (1-36). 

## More resources:
Info about the ECG processing function(ecg_clean) from Neurokit: 
https://neuropsychology.github.io/NeuroKit/functions/ecg.html#neurokit2.ecg.ecg_clean

To understand all the columns in the output: 
https://neuropsychology.github.io/NeuroKit/codebook.html 

In [40]:
# Load NeuroKit and other useful packages (make sure they are installed)
import neurokit2 as nk
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

# subject ID's to loop through 
IDs = ["49", "50", "51", "52", "54", "55", "56", "57", "58", "61", "62", "63", "65", "67",
          "68", "69", "70", "72", "73", "74", "76", "78", "81", "82", "84",
       "85", "87", "88", "89", "91", "93", "98", "99", "100", "104", "107", "109", "110"]
 
runs = [ 1, 2, 3, 4, 5, 6, 7, 8, 9]

#input_dir = 
input_dir = "/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/"


# output directory 
# outdir = "/Users/nadezhdabarbashova/Desktop/fmcc_timing/neurokit/"
outdir = "/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed/"

# plot the heart rate - it creates pretty funny plots where you can see literally evety heart beat.  
# Not the most useful but nice to double check the data for noise or disrupted signal 
figs_dir = "/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/figs/"


# Relevant info
# EVENT 
# event_chunk_countdown 
# event_type 

# the sampling rate is now 10 Hz (it was downsample by 20 in the hr_pycharm script 
# and again by 10 in the event_code script)

In [42]:
import neurokit2 as nk
import os
import pandas as pd
import numpy as np
from itertools import groupby

### process timing files (txt files)
for ID in IDs:
    for run in runs:
        try:
            #Load data
            run = str(run)
            fn = os.path.join(input_dir, f"{ID}_run{run}.txt")
            
            print(fn)
            # something gores wrong here - this line
            data = pd.read_csv(fn, sep="\t", header=None)
            #print(data.head())  # Print the first few rows to verify the output
            print("data loaded")
            data.columns = ["timepoint", "ECG", "EVENT"]
            
            # Clean data - nk package smooths the data - downsample 100
            data_cleaned = nk.ecg_clean(data["ECG"], sampling_rate = 100)

            # Get heart rate - find peaks
            signals, infoECG = nk.ecg_process(data_cleaned, sampling_rate = 100)
            
            # nk.signal_plot(ecg_cleaned)
 
            # Merge signals and data info - use peaks
            datasignal = pd.concat([signals, data], axis=1)
 
            # Save the current dataframe
            save = ID + "_EKG_rough_data_" + run + ".csv"
            #datasignal.to_csv(os.path.join(outdir, save), index=None, sep=',')
            #print("Full data with all rows exported as CSV")
            print("starting event code labelling")

            # rename df for next step. just keep it clean. 
            data = datasignal 
            
            # create new columns  
            data['event_chunk_countdown'] = 0
            data['event_chunk_flanker'] = 0
            data['threat_type'] = ""
            data['distance_type'] = ""
            data['event_type'] = ""
            
            # create new columns  pairs - start and end of countdown and start and end of flanker task 
            event_pairs_countdown = [(1, 2), (5, 6), (9, 10), (13, 14)]
            event_pairs_flanker = [(3, 4), (7, 8), (11, 12), (15, 16)]
            
            # If EVENT is an end code, assign the correct event_type
            data.loc[data['EVENT'] == 2, 'event_type'] = 'distal shock countdown end'
            data.loc[data['EVENT'] == 6, 'event_type'] = 'proximal shock countdown end'
            data.loc[data['EVENT'] == 10, 'event_type'] = 'distal stim countdown end'
            data.loc[data['EVENT'] == 14, 'event_type'] = 'proximal stim countdown end'

            print("check end codes")
            print(data[data['event_type'].notna() & data['event_type'].str.contains("end")])

            # check whether both codes in event_pairs_countdown are found in the EVENT column. 
            # find the first row index of each. 
            # create boolean mask for all rows between start and end, inclusive. 

            print("about to convert the event chunk coundown col to object")
            data['event_chunk_countdown'] = data['event_chunk_countdown'].astype("object")


            print("about to label start, end, baseline")
            for start, end in event_pairs_countdown:
                if (data['EVENT'] == start).any() and (data['EVENT'] == end).any():
                    # find the first row index of each. 
                    start_idx = data.index[data['EVENT'] == start][0] # 
                    end_idx = data.index[data['EVENT'] == end][0]
                    # create a mask - greater than start, less than end 
                    mask = (data.index >= start_idx) & (data.index <= end_idx + 1)  # Inclusive
                    data.loc[mask, 'event_chunk_countdown'] = start
                    baseline_start = max(0, start_idx - 100)
                    baseline_mask = (data.index >= baseline_start) & (data.index < start_idx)
                    data.loc[baseline_mask, 'event_type'] = "baseline"
                    data.loc[baseline_mask, 'event_chunk_countdown'] = f"baseline_{start}"

            for start, end in event_pairs_flanker:
                if (data['EVENT'] == start).any() and (data['EVENT'] == end).any():
                    start_idx = data.index[data['EVENT'] == start][0]
                    end_idx = data.index[data['EVENT'] == end][0]
                    mask = (data.index >= start_idx) & (data.index <= end_idx)  # Inclusive
                    data.loc[mask, 'event_chunk_flanker'] = start
       
            # interval_counter = 1
            # data["intervalNum"] = pd.NA  # initialize with NA to fill only relevant chunks

            # for start, end in event_pairs_countdown:
            #     start_indices = data.index[data['EVENT'] == start].tolist()
            #     end_indices = data.index[data['EVENT'] == end].tolist()

            #     # Pair start and end indices in order
            #     for s_idx in start_indices:
            #     # Find the first end index that comes after the current start index
            #         e_idx = next((e for e in end_indices if e > s_idx), None)
            #         if e_idx is not None:
            #         # Mark all rows between start and end with the current interval number
            #             mask = (data.index >= s_idx) & (data.index <= e_idx)
            #             data.loc[mask, "intervalNum"] = interval_counter
            #             interval_counter += 1

            # Now add additional labels. Mark the threat type and temporal distance type in 2 new cols 
            data.loc[data['event_chunk_countdown'] == 1, 'threat_type'] = "shock"
            data.loc[data['event_chunk_countdown'] == 1, 'distance_type'] = "distal"
            data.loc[data['event_chunk_countdown'] == 1, 'event_type'] = "distal shock countdown"
            data.loc[data['event_chunk_flanker'] == 3, 'threat_type'] = "shock"
            data.loc[data['event_chunk_flanker'] == 3, 'distance_type'] = "distal"
            data.loc[data['event_chunk_flanker'] == 3, 'event_type'] = "distal shock flanker"
            data.loc[data['event_chunk_countdown'] == 5, 'threat_type'] = "shock"
            data.loc[data['event_chunk_countdown'] == 5, 'distance_type'] = "proximal"
            data.loc[data['event_chunk_countdown'] == 5, 'event_type'] = "proximal shock countdown"
            data.loc[data['event_chunk_flanker'] == 7, 'threat_type'] = "shock"
            data.loc[data['event_chunk_flanker'] == 7, 'distance_type'] = "proximal"
            data.loc[data['event_chunk_flanker'] == 7, 'event_type'] = "proximal shock flanker"
            data.loc[data['event_chunk_countdown'] == 9, 'threat_type'] = "stim"
            data.loc[data['event_chunk_countdown'] == 9, 'distance_type'] = "distal"
            data.loc[data['event_chunk_countdown'] == 9, 'event_type'] = "distal stim countdown"
            data.loc[data['event_chunk_flanker'] == 11, 'threat_type'] = "stim"
            data.loc[data['event_chunk_flanker'] == 11, 'distance_type'] = "distal"
            data.loc[data['event_chunk_flanker'] == 11, 'event_type'] = "distal stim flanker"
            data.loc[data['event_chunk_countdown'] == 13, 'threat_type'] = "stim"
            data.loc[data['event_chunk_countdown'] == 13, 'distance_type'] = "proximal"
            data.loc[data['event_chunk_countdown'] == 13, 'event_type'] = "proximal stim countdown"
            data.loc[data['event_chunk_flanker'] == 15, 'threat_type'] = "stim"
            data.loc[data['event_chunk_flanker'] == 15, 'distance_type'] = "proximal"
            data.loc[data['event_chunk_flanker'] == 15, 'event_type'] = "proximal stim flanker"
            data.loc[data['EVENT'] == 2, 'event_type'] = 'distal shock countdown end'
            data.loc[data['EVENT'] == 6, 'event_type'] = 'proximal shock countdown end'
            data.loc[data['EVENT'] == 10, 'event_type'] = 'distal stim countdown end'
            data.loc[data['EVENT'] == 14, 'event_type'] = 'proximal stim countdown end'

            ### Check end codes 
            #print("check end codes AGAIN")
            #print(data[data['event_type'].notna() & data['event_type'].str.contains("end")])

            ## count every instance of event code.
            end_event_types = [
                'distal shock countdown end',
                'proximal shock countdown end',
                'distal stim countdown end',
                'proximal stim countdown end']


            # Initialize interval mapping and counter
            interval_mapping = {}
            interval_counter = 1

            # Create column with missing values
            data["intervalNum"] = pd.NA

            # Go row by row and mark the end event types in order
            for idx, row in data.iterrows():
                event = row['event_type']
                if event in end_event_types:
                    if event not in interval_mapping:
                        interval_mapping[event] = interval_counter
                        interval_counter += 1
                    data.at[idx, 'intervalNum'] = interval_mapping[event]

            # Backward fill the interval number across each chunk
            data["intervalNum"] = data["intervalNum"].fillna(method='bfill')

            # Filter only rows with valid event_type
            data_filtered = data[data['event_type'].notna() & (data['event_type'] != "")].copy()

            # Work on data_filtered, not data            
            print("Interval numbers:")
            print(data_filtered["intervalNum"].unique())

            # Fill any remaining NaNs in intervalNum if they exist
            if data_filtered["intervalNum"].isna().sum() > 0:
                first_valid_index = data_filtered["intervalNum"].first_valid_index()
                if first_valid_index is not None:
                    data_filtered.loc[:first_valid_index, "intervalNum"] = data_filtered.loc[first_valid_index, "intervalNum"]
            
            print("Interval numbers:")
            print(data_filtered["intervalNum"].unique())

            # *************** BASELINE CORRECTION STEP ****************
            # Create a new column for baseline-corrected ECG
            data_filtered["ECG_Baseline_Corrected"] = pd.NA

            # Loop through each interval and compute baseline using rows labeled "baseline"
            for int_num in data_filtered["intervalNum"].dropna().unique():
                int_rows = data_filtered[data_filtered["intervalNum"] == int_num]
                baseline_rows = int_rows[int_rows["event_type"] == "baseline"]

                if not baseline_rows.empty:
                    baseline_mean = baseline_rows["ECG_Rate"].mean()
                    data_filtered.loc[data_filtered["intervalNum"] == int_num, "ECG_Baseline_Corrected"] = (
                        data_filtered.loc[data_filtered["intervalNum"] == int_num, "ECG_Rate"] - baseline_mean
                    )
                else:
                    print(f"No baseline rows found for interval {int_num}. Skipping correction.")
                    data_filtered.loc[data_filtered["intervalNum"] == int_num, "ECG_Baseline_Corrected"] = (
                        data_filtered.loc[data_filtered["intervalNum"] == int_num, "ECG_Rate"]
                    )

            print("Baseline correction complete for all intervals.")


            # ** Downsample: Keep every 10th row**
            print(f"Before downsampling: {data_filtered.shape[0]} rows")
            data_downsampled = data_filtered.iloc[::10, :]
            print(f"After downsampling: {data_downsampled.shape[0]} rows")

            # Define the set of end event codes
            end_event_codes = [2, 6, 10, 14]  # These are the EVENT values for "end" rows
           
            # Remove rows with those end codes
            data_downsampled = data_downsampled[~data_downsampled["EVENT"].isin(end_event_codes)]
            print(f"After removing end codes (no longer needed): {data_downsampled.shape[0]} rows remaining")
           
           
           ## remove extra rows with countdown (the ones that appear at a start of a countdown - i.e proximal)
            # Get mask for countdown rows
            is_countdown = data_downsampled['event_type'].str.contains("countdown", na=False)

            # Create list to mark which countdown groups are "too short"
            keep_mask = np.ones(len(data_downsampled), dtype=bool)

            # Loop through groups of consecutive rows
            idx_start = 0
            for key, group in groupby(is_countdown):
                group_len = len(list(group))
                idx_end = idx_start + group_len
                if key is True and group_len < 3:
                    keep_mask[idx_start:idx_end] = False  # mark short countdown groups for removal
                idx_start = idx_end

            # Apply the mask to keep only valid rows
            data_downsampled = data_downsampled[keep_mask].reset_index(drop=True)


            # ** Save the processed dataframe**
            save_csv = os.path.join(outdir, f"{ID}_EKG_events_run{run}.csv")
            data_downsampled.to_csv(save_csv, index=False, sep=',')
            print(f"Saved: {save_csv}")
            print(f"Finished loop for: {ID}, run: {run}.")


        except Exception as e:
            print(f"Error in processing loop for: {ID}, run: {run}. Error: {e}")




/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/49_run1.txt
data loaded
starting event code labelling
check end codes
        ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
8287   0.261062   0.251982  87.331859     0.949034            0            0   
15810  0.395127   0.367151  79.211296     0.988551            0            0   
23277 -0.060846  -0.075467  70.369222     0.992094            0            0   
30692  0.049168   0.055157  63.703851     0.907943            0            0   

       ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  ...  \
8287              0              0            0             0  ...   
15810             0              0            0             0  ...   
23277             0              0            0             0  ...   
30692             0              0            0             0  ...   

       ECG_Phase_Ventricular  ECG_Phase_Completion_Ventricular  timepoint  \
8287                     1.0        

Interval numbers:
[1 2 3 4 <NA>]
Interval numbers:
[1 2 3 4 <NA>]
Baseline correction complete for all intervals.
Before downsampling: 24417 rows
After downsampling: 2442 rows
After removing end codes: 2441 rows remaining
Saved: /Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed/49_EKG_events_run4.csv
Finished loop for: 49, run: 4.
/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/49_run5.txt
data loaded
starting event code labelling
check end codes
        ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
8998   0.188497   0.179227  78.068643     0.762594            0            1   
16436  0.091389   0.082094  95.582694     0.656556            0            0   
23963 -0.088946  -0.072550  59.545937     0.557078            0            0   
31421  0.486388   0.489315  66.399012     0.720102            0            0   

       ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  ...  \
8998              0              0            0  

Interval numbers:
[1 2 3 4 <NA>]
Interval numbers:
[1 2 3 4 <NA>]
Baseline correction complete for all intervals.
Before downsampling: 24420 rows
After downsampling: 2442 rows
After removing end codes: 2442 rows remaining
Saved: /Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed/49_EKG_events_run8.csv
Finished loop for: 49, run: 8.
/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/49_run9.txt
data loaded
starting event code labelling
check end codes
        ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
10198  0.063747   0.066625  54.331242     0.431163            0            0   
17653  0.280504   0.282659  82.675933     0.823532            0            0   
25269 -0.243949  -0.222934  78.454403     0.777004            0            0   
32685 -0.080940  -0.095340  73.392370     0.880487            0            0   

       ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  ...  \
10198             0              0            0  

Interval numbers:
[1 2 3 4 <NA>]
Interval numbers:
[1 2 3 4 <NA>]
Baseline correction complete for all intervals.
Before downsampling: 24418 rows
After downsampling: 2442 rows
After removing end codes: 2442 rows remaining
Saved: /Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed/50_EKG_events_run3.csv
Finished loop for: 50, run: 3.
/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/50_run4.txt
data loaded
starting event code labelling
check end codes
        ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
8688  -0.156220  -0.160354  73.220316     0.828017            0            0   
16083 -0.086769  -0.088411  72.529814     0.872535            0            0   
23458 -0.205981  -0.198587  76.041099     0.919863            0            0   
30880 -0.145545  -0.142189  62.577446     0.516768            0            0   

       ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  ...  \
8688              0              0            0  

Interval numbers:
[1 2 3 4 <NA>]
Interval numbers:
[1 2 3 4 <NA>]
Baseline correction complete for all intervals.
Before downsampling: 24415 rows
After downsampling: 2442 rows
After removing end codes: 2442 rows remaining
Saved: /Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed/50_EKG_events_run7.csv
Finished loop for: 50, run: 7.
/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/50_run8.txt
data loaded
starting event code labelling
check end codes
        ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
9945   0.253053   0.256724  82.312229     0.949169            0            0   
17444  0.129552   0.122505  77.615805     0.972904            0            0   
24983 -0.267092  -0.253468  64.003320     0.703778            0            0   
32369 -0.137182  -0.102138  69.229890     0.884254            0            0   

       ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  ...  \
9945              0              0            0  

...

Interval numbers:
[1 2 3 4 <NA>]
Interval numbers:
[1 2 3 4 <NA>]
Baseline correction complete for all intervals.
Before downsampling: 24415 rows
After downsampling: 2442 rows
After removing end codes: 2441 rows remaining
Saved: /Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed/107_EKG_events_run9.csv
Finished loop for: 107, run: 9.
/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/109_run1.txt
data loaded
starting event code labelling
check end codes
        ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
7212  -0.176222  -0.168756  85.675399     0.722387            0            0   
14934 -0.131558  -0.130897  80.570712     0.752793            0            0   
22535  0.539505   0.519162  95.751146     0.683758            0            0   
30248 -0.130863  -0.125991  72.953913     0.875621            0            0   

       ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  ...  \
7212              0              0            

Interval numbers:
[1 2 3 4 <NA>]
Interval numbers:
[1 2 3 4 <NA>]
Baseline correction complete for all intervals.
Before downsampling: 24416 rows
After downsampling: 2442 rows
After removing end codes: 2442 rows remaining
Saved: /Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed/109_EKG_events_run4.csv
Finished loop for: 109, run: 4.
/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/109_run5.txt
data loaded
starting event code labelling
check end codes
        ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
7389  -0.382770  -0.379737  73.436304     0.668398            0            0   
14636 -0.112079  -0.109519  74.443295     0.966329            0            0   
22156  0.623723   0.603401  83.451776     0.883557            0            0   
29419 -0.184645  -0.187454  72.633782     0.699350            0            0   

       ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  ...  \
7389              0              0            

Interval numbers:
[1 2 3 4 <NA>]
Interval numbers:
[1 2 3 4 <NA>]
Baseline correction complete for all intervals.
Before downsampling: 24414 rows
After downsampling: 2442 rows
After removing end codes: 2442 rows remaining
Saved: /Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed/109_EKG_events_run8.csv
Finished loop for: 109, run: 8.
/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/109_run9.txt
data loaded
starting event code labelling
check end codes
        ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
9728  -0.208473  -0.205392  94.433819     0.908589            0            0   
17055 -0.027064  -0.027755  85.405134     0.912419            0            0   
24426  0.124487   0.125147  75.090291     0.630819            0            0   
31764  0.120424   0.127812  81.692277     0.816925            0            0   

       ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  ...  \
9728              0              0            

Interval numbers:
[1 2 3 4 <NA>]
Interval numbers:
[1 2 3 4 <NA>]
Baseline correction complete for all intervals.
Before downsampling: 24418 rows
After downsampling: 2442 rows
After removing end codes: 2441 rows remaining
Saved: /Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed/110_EKG_events_run3.csv
Finished loop for: 110, run: 3.
/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/110_run4.txt
data loaded
starting event code labelling
check end codes
        ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
8637   0.000871   0.000760  12.338082     0.873810            0            0   
16013  0.000408   0.000327  17.400677     0.968366            0            0   
23304  0.000367   0.000714  20.946793     0.643187            0            0   
30605  0.000252   0.000355  10.101443     0.977216            0            0   

       ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  ...  \
8637              0              0            

Interval numbers:
[1 2 3 4 <NA>]
Interval numbers:
[1 2 3 4 <NA>]
Baseline correction complete for all intervals.
Before downsampling: 24414 rows
After downsampling: 2442 rows
After removing end codes: 2442 rows remaining
Saved: /Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed/110_EKG_events_run7.csv
Finished loop for: 110, run: 7.
/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/timing_files/110_run8.txt
data loaded
starting event code labelling
check end codes
        ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
7918   0.002715   0.002361  16.113458     0.911473            0            0   
15262  0.001453   0.001390  15.095608     1.053677            0            0   
22732  0.001419   0.001124  22.233968     0.698242            0            0   
30235  0.004404   0.003951  43.478261     0.000000            0            0   

       ECG_P_Onsets  ECG_P_Offsets  ECG_Q_Peaks  ECG_R_Onsets  ...  \
7918              0              0            

## Combine Files

The block below will loop thorugh the CSV files and combine them all (all subjects and all runs) into a single, final CSV file. 


In [43]:
import os
import glob
import pandas as pd

# Where the input files are
combined_dir = "/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/processed"
pattern = os.path.join(combined_dir, "*_EKG_events_run*.csv")

# Where to save the final combined CSV
output_dir = "/Users/nadezhdabarbashova/Documents/fmcc_heart_rate/final"
os.makedirs(output_dir, exist_ok=True)

all_csv_files = glob.glob(pattern)

print(f"🔍 Found {len(all_csv_files)} matching files.")
all_dfs = []

for file in all_csv_files:
    df = pd.read_csv(file)

    # Extract run number
    run_str = os.path.basename(file).split("_run")[1].split(".csv")[0]
    run_num = int(run_str)
    df["run"] = run_num

    # Compute countdownNum if intervalNum exists
    if "intervalNum" in df.columns:
        df["countdownNum"] = (df["run"] - 1) * 4 + df["intervalNum"]
    else:
        df["countdownNum"] = pd.NA

    all_dfs.append(df)

# Combine and save
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_csv_path = os.path.join(output_dir, "EKG_combined_all_participants.csv")
combined_df.to_csv(combined_csv_path, index=False)

print(f"✅ Combined {len(all_csv_files)} files with {combined_df.shape[0]} rows.")
print(f"💾 Saved to: {combined_csv_path}")


🔍 Found 351 matching files.
✅ Combined 351 files with 844745 rows.
💾 Saved to: /Users/nadezhdabarbashova/Documents/fmcc_heart_rate/final/EKG_combined_all_participants.csv


In [ ]:
# look at one example of the output 
# understand these cols: 
# examine these columns 
# ECG_Raw, ECG_Clean, ECG_Rate, ECG_Quality timepoint	EVENT	Index	countdownNum	ECG_Baseline_Corrected



In [ ]:
# keep this as a reference - alternative way to baseline correct - use epoching 
# example code
eventIndex = events["onset"]
eventList = [] 

for i in eventIndex: 
    d = df1["Event"][i]
    eventList.append(d)

for epoch_index in epoch4: 
    df[epoch_index] = {} 
    epoch = epochs4[epoch_index]
    ecg_baseline = epoch["hr"].loc[-2000:0].mean() 
    ecg_mean = epoch["hr"].loc[0:8000].mean()
    # store ECG in df 
    df[epoch_index]["hr"] = ecg_mean - ecg_baseline 

df = pd.DataFrame.from_dict(df, orient = "index")
df["Event"] = eventList 
df 


In [ ]:
# original sampling rate was 2000 - but has been downsampled by 20
# sampling rate is 100
# 30 seconds = 100 * 30 = 3000 rows 

# create a basline 1 second before
# then create a mean of the next 30 seconds
# not sure what hr is 

for epoch_index in epoch4: 
    df[epoch_index] = {} 
    epoch = epochs4[epoch_index]
    ecg_baseline = epoch["hr"].loc[-100:0].mean()  # change to 100  
    ecg_mean = epoch["hr"].loc[0:3000].mean() # this should be 30 seconds 
    df[epoch_index]["hr"] = ecg_mean - ecg_baseline 

